### Manipulation of PeaskDB de novo-assisted database search results of _T. weisflogii_ rot samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Multiple injections and fragmentation strategies included
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [2]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_331-T5nd-all_PEAKS_99/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_331-T5nd-all_PEAKS_99


In [3]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [4]:
ls 

331-T5nd-all_PEAKS_99_DB-search-psm.csv
331-T5nd-all_PEAKS_99_DNO.csv
331-T5nd-all_PEAKS_99_peptide.csv
331-T5nd-all_PEAKS_99_peptides_1_0_0.mzid
331-T5nd-all_PEAKS_99_peptides.pep.xml
331-T5nd-all_PEAKS_99_protein-peptides.csv
331-T5nd-all_PEAKS_99_proteins.csv
331-T5nd-all_PEAKS_99_proteins.fasta


In [6]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup331 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_331-T5nd-all_PEAKS_99/331-T5nd-all_PEAKS_99_peptide.csv")

# remove redundant rows
peaksdb331 = pd.DataFrame.drop_duplicates(peaksdbdup331)

print(peaksdb331.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb331.columns = columns

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup331))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb331))

#look at the dataframe
peaksdb331.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 331_T5_nodigest', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec 331_T5_nodigest', 'Accession', 'PTM', 'AScore'],
      dtype='object')
# redundant peaksdb peptides in combined dataframe 235
# nonredundant peaksdb peptides in combined dataframe 235


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source File,#Spec,#Spec,Accession,PTM,AScore
0,LPQVEGTGGDVQPSQDLVR,39.68,1994.0068,19,1.9,665.6775,76.39,173000000.0,21,15802,20210115_Weissrot_331_T5_nodigest_EThcD_120min...,12,12,gi|54036848,NaN,NaN
1,VIGQNEAVDAVSNAIR,33.19,1654.8638,16,0.7,552.6289,93.08,158000000.0,21,19044,20210115_Weissrot_331_T5_nodigest_EThcD_120min...,24,24,gi|54036848,NaN,NaN
2,AIDLIDEAASSIR,30.10,1372.7197,13,2.8,458.5818,109.95,135000000.0,21,22501,20210115_Weissrot_331_T5_nodigest_EThcD_120min...,23,23,gi|54036848,NaN,NaN
3,VTDAEIAEVLAR,26.56,1285.6877,12,0.7,643.8516,98.08,67900000.0,20,31248,20210114_Weissrot_331_T5_nodigest_DDA_120min_2...,17,17,gi|54036848,NaN,NaN
4,TGLIPHPVPL,25.63,1042.6174,10,1.6,522.3168,97.06,848000.0,20,30838,20210114_Weissrot_331_T5_nodigest_DDA_120min_2...,2,2,Thalassiosira_weissflogii_0201805882,NaN,NaN


In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb331['A'] = peaksdb331['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb331['C'] = peaksdb331['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb331['D'] = peaksdb331['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb331['E'] = peaksdb331['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb331['F'] = peaksdb331['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb331['G'] = peaksdb331['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb331['H'] = peaksdb331['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb331 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb331['I'] = peaksdb331['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb331['K'] = peaksdb331['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb331['L'] = peaksdb331['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb331['M'] = peaksdb331['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb331['N'] = peaksdb331['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb331['P'] = peaksdb331['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb331['Q'] = peaksdb331['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb331['R'] = peaksdb331['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb331['S'] = peaksdb331['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb331['T'] = peaksdb331['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb331['V'] = peaksdb331['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb331['W'] = peaksdb331['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb331['Y'] = peaksdb331['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb331['c-carb'] = peaksdb331['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb331['m-oxid'] = peaksdb331['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaksdb331['k-oxid'] = peaksdb331['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaksdb331['p-oxid'] = peaksdb331['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaksdb331['r-oxid'] = peaksdb331['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaksdb331['y-oxid'] = peaksdb331['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb331['n-deam'] = peaksdb331['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaksdb331['k-meth'] = peaksdb331['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb331['r-meth'] = peaksdb331['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaksdb331['q-pyro'] = peaksdb331['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb331['k-acet'] = peaksdb331['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb331['stripped peptide'] = peaksdb331['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb331['stripped length'] = peaksdb331['stripped peptide'].apply(len)

peaksdb331['NAAF num.'] = peaksdb331['Area'] / peaksdb331['stripped length']

# total the number of modifications in sequence
peaksdb331['ptm-total'] = peaksdb331['c-carb'] + peaksdb331['m-oxid'] + peaksdb331['k-oxid'] + peaksdb331['p-oxid'] \
+ peaksdb331['r-oxid'] + peaksdb331['y-oxid'] + peaksdb331['n-deam'] + peaksdb331['k-meth'] + peaksdb331['r-meth'] \
+ peaksdb331['q-pyro'] + peaksdb331['k-acet']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb331['stripped I-L']= peaksdb331['stripped peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb331.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1.csv")

# check out the results
peaksdb331.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped peptide,stripped length,NAAF num.,ptm-total,stripped I-L
0,LPQVEGTGGDVQPSQDLVR,39.68,1994.0068,19,1.9,665.6775,76.39,173000000.0,21,15802,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,9.105263e+06,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,33.19,1654.8638,16,0.7,552.6289,93.08,158000000.0,21,19044,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,9.875000e+06,0,VLGQNEAVDAVSNALR
2,AIDLIDEAASSIR,30.10,1372.7197,13,2.8,458.5818,109.95,135000000.0,21,22501,...,0,0,0,0,0,AIDLIDEAASSIR,13,1.038462e+07,0,ALDLLDEAASSLR
3,VTDAEIAEVLAR,26.56,1285.6877,12,0.7,643.8516,98.08,67900000.0,20,31248,...,0,0,0,0,0,VTDAEIAEVLAR,12,5.658333e+06,0,VTDAELAEVLAR
4,TGLIPHPVPL,25.63,1042.6174,10,1.6,522.3168,97.06,848000.0,20,30838,...,0,0,0,0,0,TGLIPHPVPL,10,8.480000e+04,0,TGLLPHPVPL


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep331moddup = peaksdb331[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep331dup = peaksdb331[["stripped peptide"]]

# deduplicate both of these lists
pep331mod = pep331moddup.drop_duplicates()
pep331 = pep331dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 331:", len(pep331moddup))
print("Deduplicated modified PeaksDB peptides in 331:", len(pep331mod))
print("Total stripped PeaksDB peptides in 331:", len(pep331dup))
print("Deduplicated stripped PeaksDB peptides in 331:", len(pep331))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep331.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.fas

# look at the stripped peptides
pep331.head()

Total modified PeaksDB peptides in 331: 235
Deduplicated modified PeaksDB peptides in 331: 235
Total stripped PeaksDB peptides in 331: 235
Deduplicated stripped PeaksDB peptides in 331: 219


,stripped peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AIDLIDEAASSIR
3,VTDAEIAEVLAR
4,TGLIPHPVPL


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb331['A'].sum(),
        'C': peaksdb331['C'].sum(),
        'D': peaksdb331['D'].sum(),
        'E': peaksdb331['E'].sum(),
        'F': peaksdb331['F'].sum(),
        'G': peaksdb331['G'].sum(),
        'H': peaksdb331['H'].sum(),
        'I': peaksdb331['I'].sum(),
        'K': peaksdb331['K'].sum(),
        'L': peaksdb331['L'].sum(),
        'M': peaksdb331['M'].sum(),
        'N': peaksdb331['N'].sum(),
        'P': peaksdb331['P'].sum(),
        'Q': peaksdb331['Q'].sum(),
        'R': peaksdb331['R'].sum(),
        'S': peaksdb331['S'].sum(),
        'T': peaksdb331['T'].sum(),
        'V': peaksdb331['V'].sum(),
        'W': peaksdb331['W'].sum(),
        'Y': peaksdb331['Y'].sum(),
        'c-carb': peaksdb331['c-carb'].sum(),
        'm-oxid': peaksdb331['m-oxid'].sum(),
        'k-oxid': peaksdb331['k-oxid'].sum(),
        'p-oxid': peaksdb331['p-oxid'].sum(),
        'r-oxid': peaksdb331['r-oxid'].sum(),
        'y-oxid': peaksdb331['y-oxid'].sum(),
        'n-deam': peaksdb331['n-deam'].sum(),
        'k-meth': peaksdb331['k-meth'].sum(),
        'r-meth': peaksdb331['r-meth'].sum(),
        'q-pyro': peaksdb331['q-pyro'].sum(),
        'k-acet': peaksdb331['k-acet'].sum(),
        'Total area': peaksdb331['Area'].sum(),
        'Total length': peaksdb331['stripped length'].sum()
       }

totalpeaksdb331 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                              'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                              'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb331['% C w/ carb'] = totalpeaksdb331['c-carb'] / totalpeaksdb331['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb331['% M w/ oxid'] = totalpeaksdb331['m-oxid'] / totalpeaksdb331['M'] 

# calculate percentage of K's that are oxidized
totalpeaksdb331['% K w/ oxid'] = totalpeaksdb331['k-oxid'] / totalpeaksdb331['K'] 

# calculate percentage of P's that are oxidized
totalpeaksdb331['% P w/ oxid'] = totalpeaksdb331['p-oxid'] / totalpeaksdb331['P'] 

# calculate percentage of R's that are oxidized
totalpeaksdb331['% R w/ oxid'] = totalpeaksdb331['p-oxid'] / totalpeaksdb331['R'] 

# calculate percentage of Y's that are oxidized
totalpeaksdb331['% Y w/ oxid'] = totalpeaksdb331['y-oxid'] / totalpeaksdb331['Y'] 

# calculate percentage of N's that are deamidated
totalpeaksdb331['% N w/ deam'] = totalpeaksdb331['n-deam'] / totalpeaksdb331['N'] 

# calculate percentage of K's that are methylated
totalpeaksdb331['% K w/ meth'] = totalpeaksdb331['k-meth'] / totalpeaksdb331['K'] 

# calculate percentage of R's that are methylated
totalpeaksdb331['% R w/ meth'] = totalpeaksdb331['r-meth'] / totalpeaksdb331['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaksdb331['% Q w/ pyro'] = totalpeaksdb331['q-pyro'] / totalpeaksdb331['Q'] 

# calculate percentage of K's that are acetylation
totalpeaksdb331['% K w/ acet'] = totalpeaksdb331['k-acet'] / totalpeaksdb331['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb331['NAAF denom.'] = totalpeaksdb331['Total area'] / totalpeaksdb331['Total length']

# write modified dataframe to new txt file
totalpeaksdb331.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_totals.csv")

totalpeaksdb331.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,115,1,263,170,93,325,59,166,78,202,...,0.0,0.014388,0.075472,0.025,0.020408,0.0,0.0,0.050847,0.0,1.252107e+06


In [10]:
# use the calculated NAAF factor (in totalpeaksdb331 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 1.252107e+06

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb331['NAAF factor'] = (peaksdb331['NAAF num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb331_NAAF = peaksdb331[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', \
                              'k-meth', 'r-meth', 'q-pyro', 'k-acet']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb331_NAAF['A-NAAF20'] = peaksdb331_NAAF['A'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['C-NAAF20'] = peaksdb331_NAAF['C'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['D-NAAF20'] = peaksdb331_NAAF['D'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['E-NAAF20'] = peaksdb331_NAAF['E'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['F-NAAF20'] = peaksdb331_NAAF['F'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['G-NAAF20'] = peaksdb331_NAAF['G'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['H-NAAF20'] = peaksdb331_NAAF['H'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['I-NAAF20'] = peaksdb331_NAAF['I'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['K-NAAF20'] = peaksdb331_NAAF['K'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['L-NAAF20'] = peaksdb331_NAAF['L'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['M-NAAF20'] = peaksdb331_NAAF['M'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['N-NAAF20'] = peaksdb331_NAAF['N'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['P-NAAF20'] = peaksdb331_NAAF['P'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['Q-NAAF20'] = peaksdb331_NAAF['Q'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['R-NAAF20'] = peaksdb331_NAAF['R'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['S-NAAF20'] = peaksdb331_NAAF['S'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['T-NAAF20'] = peaksdb331_NAAF['T'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['V-NAAF20'] = peaksdb331_NAAF['V'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['W-NAAF20'] = peaksdb331_NAAF['W'] * peaksdb331['NAAF factor']
peaksdb331_NAAF['Y-NAAF20'] = peaksdb331_NAAF['Y'] * peaksdb331['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb331_NAAF['ccarb-NAAF20'] = peaksdb331_NAAF['c-carb'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['moxid-NAAF20'] = peaksdb331_NAAF['m-oxid'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['koxid-NAAF20'] = peaksdb331_NAAF['k-oxid'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['poxid-NAAF20'] = peaksdb331_NAAF['p-oxid'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['roxid-NAAF20'] = peaksdb331_NAAF['r-oxid'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['yoxid-NAAF20'] = peaksdb331_NAAF['y-oxid'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['ndeam-NAAF20'] = peaksdb331_NAAF['n-deam'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['kmeth-NAAF20'] = peaksdb331_NAAF['k-meth'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['rmeth-NAAF20'] = peaksdb331_NAAF['r-meth'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['qpyro-NAAF20'] = peaksdb331_NAAF['q-pyro'] * peaksdb331_NAAF['NAAF factor']
peaksdb331_NAAF['kacet-NAAF20'] = peaksdb331_NAAF['k-acet'] * peaksdb331_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb331_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_NAAF.csv")

peaksdb331_NAAF.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,moxid-NAAF20,koxid-NAAF20,poxid-NAAF20,roxid-NAAF20,yoxid-NAAF20,ndeam-NAAF20,kmeth-NAAF20,rmeth-NAAF20,qpyro-NAAF20,kacet-NAAF20
0,LPQVEGTGGDVQPSQDLVR,7.271953,0,0,2,1,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,VIGQNEAVDAVSNAIR,7.886706,3,0,1,1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AIDLIDEAASSIR,8.293712,3,0,2,1,0,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,VTDAEIAEVLAR,4.519049,3,0,1,2,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TGLIPHPVPL,0.067726,0,0,0,0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb331 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb331_NAAF['NAAF factor'].sum(),
        'A': peaksdb331_NAAF['A'].sum(),
        'C': peaksdb331_NAAF['C'].sum(),
        'D': peaksdb331_NAAF['D'].sum(),
        'E': peaksdb331_NAAF['E'].sum(),
        'F': peaksdb331_NAAF['F'].sum(),
        'G': peaksdb331_NAAF['G'].sum(),
        'H': peaksdb331_NAAF['H'].sum(),
        'I': peaksdb331_NAAF['I'].sum(),
        'K': peaksdb331_NAAF['K'].sum(),
        'L': peaksdb331_NAAF['L'].sum(),
        'M': peaksdb331_NAAF['M'].sum(),
        'N': peaksdb331_NAAF['N'].sum(),
        'P': peaksdb331_NAAF['P'].sum(),
        'Q': peaksdb331_NAAF['Q'].sum(),
        'R': peaksdb331_NAAF['R'].sum(),
        'S': peaksdb331_NAAF['S'].sum(),
        'T': peaksdb331_NAAF['T'].sum(),
        'V': peaksdb331_NAAF['V'].sum(),
        'W': peaksdb331_NAAF['W'].sum(),
        'Y': peaksdb331_NAAF['Y'].sum(),
        'c-carb': peaksdb331_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb331_NAAF['moxid-NAAF20'].sum(),
        'k-oxid': peaksdb331_NAAF['koxid-NAAF20'].sum(),
        'p-oxid': peaksdb331_NAAF['poxid-NAAF20'].sum(),
        'r-oxid': peaksdb331_NAAF['roxid-NAAF20'].sum(),
        'y-oxid': peaksdb331_NAAF['yoxid-NAAF20'].sum(),
        'n-deam': peaksdb331_NAAF['ndeam-NAAF20'].sum(),
        'k-meth': peaksdb331_NAAF['kmeth-NAAF20'].sum(),
        'r-meth': peaksdb331_NAAF['rmeth-NAAF20'].sum(),
        'q-pyro': peaksdb331_NAAF['qpyro-NAAF20'].sum(),
        'k-acet': peaksdb331_NAAF['kacet-NAAF20'].sum()
       }

totalpeaksdb331_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                                   'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                                   'q-pyro', \
                                                   'k-acet'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb331_NAAF['% C w/ carb.'] = totalpeaksdb331_NAAF['c-carb'] / totalpeaksdb331_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb331_NAAF['% M w/ oxid'] = totalpeaksdb331_NAAF['m-oxid'] / totalpeaksdb331_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaksdb331_NAAF['% K w/ oxid'] = totalpeaksdb331_NAAF['k-oxid'] / totalpeaksdb331_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaksdb331_NAAF['% P w/ oxid'] = totalpeaksdb331_NAAF['p-oxid'] / totalpeaksdb331_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaksdb331_NAAF['% R w/ oxid'] = totalpeaksdb331_NAAF['r-oxid'] / totalpeaksdb331_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaksdb331_NAAF['% Y w/ oxid'] = totalpeaksdb331_NAAF['y-oxid'] / totalpeaksdb331_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb331_NAAF['% N w/ deam'] = totalpeaksdb331_NAAF['n-deam'] / totalpeaksdb331_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb331_NAAF['% K w/ meth'] = totalpeaksdb331_NAAF['k-meth'] / totalpeaksdb331_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaksdb331_NAAF['% R w/ meth'] = totalpeaksdb331_NAAF['r-meth'] / totalpeaksdb331_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaksdb331_NAAF['% Q w/ pyro'] = totalpeaksdb331_NAAF['q-pyro'] / totalpeaksdb331_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb331_NAAF['% K w/ acet'] = totalpeaksdb331_NAAF['k-acet'] / totalpeaksdb331_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb331_NAAF['NAAF check'] = totalpeaksdb331_NAAF['NAAF'] / 1.252107e+06

# write modified dataframe to new txt file, same name + totals
totalpeaksdb331_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_331_T5_trypsin_combine_PTMopt_DB_FDR1_NAAF_totals.csv")

totalpeaksdb331_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF check
sample total,229.477809,115,1,263,170,93,325,59,166,78,...,0.0,0.000508,0.00522,0.002577,0.001089,0.0,0.0,0.001251,0.0,0.000183
